In [1]:
import os
import pandas as pd

# load all datasets in synergy_dataset
path = 'synergy_dataset'
files = os.listdir(path)
files = [f for f in files if f.endswith('.csv')]

In [ ]:
# settings generator 
for model in ["nb", "logistic", "nn-2-layer", "xgboost"]:
    for fe in ["tfidf", "doc2vec", "embedding-idf", "embedding-lstm", "reuseable_MiniLM", "reuseable_sbert"]:
        for b in ["simple", "double", "triple", "undersample"]:
            for q in ["max", "max_random", "max_uncertainty", "uncertainty", "cluster"]:


In [2]:
settings = [""]#["-m rf -e reuseable_MiniLM", "-m nb -e tfidf"]
for i, setting in enumerate(settings):
    for dataset in files:
        template = f"""apiVersion: batch/v1
kind: Job
metadata:
  name: sim-{dataset.lower().replace("_","-")[:-4]}-{i}
spec:
  template:
      metadata:
        labels:
          app: simulation
      spec:
        restartPolicy: Never
        containers:
        - name: sim
          image: ghcr.io/jteijema/asreview-simulation-project:v1.1.0
          resources:
            requests:
              memory: 4Gi
              cpu: 500m
          env:
          - name: DATASET
            value: {dataset[:-4]}
          - name: SETTINGS
            value: "{setting}"
          - name: EXOSCALE_API_KEY
            valueFrom:
              secretKeyRef:
                name: exoscale-credentials
                key: EXOSCALE_API_KEY
          - name: EXOSCALE_API_SECRET
            valueFrom:
              secretKeyRef:
                name: exoscale-credentials
                key: EXOSCALE_API_SECRET
          - name: EXOSCALE_STORAGE_ZONE
            valueFrom:
              secretKeyRef:
                name: exoscale-credentials
                key: EXOSCALE_STORAGE_ZONE
          - name: BUCKET
            valueFrom:
              secretKeyRef:
                name: exoscale-credentials
                key: BUCKET"""

        # save template to file
        with open(f"jobs/sim-{i}-{dataset.lower().replace('_','-')[:-4]}.yaml", "w") as f:
            f.write(template)
    

In [ ]:
# get all jobs
jobs = os.listdir('jobs')
for job in jobs:
    print(f"!kubectl create -f ./jobs/{job}")
    !kubectl create -f ./jobs/{job}